# Preprocessing Cheat Sheet
```PSMCV-2.2024-2025.1```

Authors: 
- Robbert van der Mijn, w.r.van.der.mijn@rug.nl, 2022
- Mark Span, m.m.span@rug.nl, 2024

See also the tutorial at: https://pydatamatrix.eu/1.0/eyelinkparser/

and definitely:

Mathôt S, Vilotijević A. Methods in cognitive pupillometry: Design, preprocessing, and statistical analysis. Behav Res Methods. 2023 Sep;55(6):3055-3077. doi: 10.3758/s13428-022-01957-7. Epub 2022 Aug 26. PMID: 36028608; PMCID: PMC10556184.

Additional References
Kahneman, D., & Beatty, J. (1966). Pupil diameter and load on memory. Science, 154(3756), 1583–1585. https://doi.org/10.1126/science.154.3756.1583
Mathôt, S., (2018). Pupillometry: Psychology, Physiology, and Function. Journal of Cognition. 1(1), p.16. https://doi.org/10.5334/joc.18

# Experiment Overview

We are conducting an experiment to analyze how subjects decide whether the provided answer to arithmetic problems is correct or incorrect. The experiment includes both simple and moderately challenging arithmetic problems. The hypothesis is that solving more difficult problems will elicit different pupil responses compared to simpler problems.

### Subjects and Task
Four subjects participated in the task. Each subject was presented with an arithmetic problem displayed on the screen, followed by a suggested answer. The subjects then responded by pressing the 'f' key if they believed the answer was incorrect, or the 'j' key if they believed the answer was correct. The objective is to measure the accuracy and response time in relation to the difficulty of the problems.

### Eyetracker Integration
To monitor the different stages of each trial, we sent specific messages to the eyetracker, which are embedded within the eyetracker data. These messages help in segmenting the data according to the phases of the trial. The sequence of phases and corresponding messages used in OpenSesame is as follows:

1. Start Trial: Initiated with the message "start_trial".
1. Baseline Phase: Marked with the message "baseline".
1. Problem Presentation: Indicated by the message "problem".
1. Fixation Phase: Marked by the message "fixation".
1. Response Phase: Denoted by the message "response".
1. Feedback Phase: Signified by the message "feedback".
1. End Trial: Concluded with the message "stop_trial".

### Data Collection
For each trial, the following data points are recorded:

Difficulty: The level of difficulty of the arithmetic problem (simple or more difficult).
Response: The key pressed by the subject ('f' or 'j').
Correctness: Whether the subject's response was correct or incorrect.

This detailed data collection allows us to analyze the relationship between problem difficulty, pupil response, and the accuracy of the subject's answers.

## Modules and Packages

This section outlines the necessary modules and packages required for the analysis of the pupil data. Ensure that all these packages are installed in your environment.

- **eyelinkparser**: A package for parsing EyeLink data files.
- **datamatrix**: A library for working with data matrices, providing functionality for data manipulation and analysis.
- **numpy**: A fundamental package for numerical computations in Python.
- **scipy**: A library used for scientific and technical computing.
- **matplotlib**: A library for plotting data


# __See Install.ipynb for the code to install these__

## Data Loading

This section covers the steps required to load the data into the notebook.

The data will be represented in python using DataMatrices. These are structures tailored to be able to contain the information that is associated with trial-based eyetracking data.

To imagine what your data should look like, we first take a step back: 

For each phase, timeseries will be created with information about fixations:

- fixxlist_[phase name] is a series of X coordinates
- fixylist_[phase name] is a series of Y coordinates
- fixstlist_[phase name] is a series of fixation start times
- fixetlist_[phase name] is a series of fixation end times

and info on the blinks:

- blinkstlist_[phase name] is a series of blink start times
- blinketlist_[phase name] is a series of blink end times

Additionally, four columns will be created with information about individual gaze samples:

- xtrace_[phase name] is a series of X coordinates
- ytrace_[phase name] is a series of Y coordinates
- ttrace_[phase name] is a series of time stamps
- ptrace_[phase name] is a series of pupil sizes

**We** are only interested in the last trace. This is the size of the pupil, in "Arbitrary Units".


In [7]:
# Datamatrix is also from Sebastiaan. You need a conveinient method to manage all your data.
# The code below is needed to use the libraries that do the hard work for us.

from datamatrix import (
    plot,
    operations as ops,
    series as srs,
    functional as fnc,
    SeriesColumn,
    convert,
)

from eyelinkparser import parse, defaulttraceprocessor
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.font_manager


## Declaring a function

The eyelinkparser needs info from you on what your trials look like (the *phases*. Therefore you need to write a function that reads in your trials and defines its structure in there. Like this:

In [8]:
def my_preprocessor():
    
    dm = parse(
        traceprocessor = defaulttraceprocessor(
          blinkreconstruct = False,
          downsample = None
        )
    )
    
    # define the max depth manually: baseline, problem, fixation, response and feedback
    MAX_DEPTH = 1500 + 800 + 2500 + 2000 

    # get the current max depth, we'll be cutting it off later
    max_depth = dm.ptrace_baseline.depth + dm.ptrace_problem.depth + dm.ptrace_fixation.depth + dm.ptrace_response.depth + dm.ptrace_feedback.depth

    # Create new series column that will hold our new pupil trace
    dm.pupil = SeriesColumn(depth = max_depth)

    # For each trial, trim pupil traces based on the nan values in the time trace (there are more elegant solutions I'm sure)
    for i, row in enumerate(dm):
        ptrace_baseline = row.ptrace_baseline[~np.isnan(row.ttrace_baseline)]
        ptrace_problem  = row.ptrace_problem[~np.isnan(row.ttrace_problem)]
        ptrace_fixation = row.ptrace_fixation[~np.isnan(row.ttrace_fixation)]
        ptrace_response = row.ptrace_response[~np.isnan(row.ttrace_response)]
        ptrace_feedback = row.ptrace_feedback[~np.isnan(row.ttrace_feedback)]
        
        # Concatenate and pad so they're all the same depth
        pupil = np.concatenate((ptrace_baseline, ptrace_problem, ptrace_fixation, ptrace_response, ptrace_feedback))
        pupil = np.pad(pupil, (0, max_depth - len(pupil)), "constant", constant_values = np.nan)
        
        # Write this trial to the original dm
        dm.pupil[i] = pupil
        
    # Trim the original dm to our manual depth (the feedback phase of some of the really long trials will be cut off)
    dm.pupil.depth = MAX_DEPTH
    # and return only the data we want to analyse. 
    dm = dm[("subject_nr", "pupil", "count_trial_sequence", "correct", "difficulty", "practice", "response_time")]
    
    return dm


Notice that running the above code does nothing yet. It defines a function that will read your data, and creates a timeseries for each trial containing the pupilsize data. Also, note that for your own experiment, you will have to supply the phases that define your trial, and the total time that a trial can take in this part of your script!

In [9]:
# Actually reading the data 

dm = my_preprocessor()


............................................................................................................................................................................................................................

In [10]:
import pygwalker as pyg
import pandas as pd
newdm = dm[("subject_nr", "count_trial_sequence", "correct", "difficulty", "practice", "response_time")]
newdm = dm
thepyg = convert.to_pandas(newdm)
thepyg['correct'] = thepyg['correct'].astype('bool')
walker = pyg.walk(thepyg)

Box(children=(HTML(value='\n<div id="ifr-pyg-000621c205577c14GLTtoQH3zWsEKxMS" style="height: auto">\n    <hea…

Again: notice that ```dm``` is a *datamatrix*. It contains all the data we need to analyse the pupil responses to each of the trial categories.

Now select the non-practice trials:

In [ ]:
dm = (dm.practice == 'no')
dm

Of course, you could also select only the correct trials here. We will not do so, so we can show you how to plot and test multiple variable designs.

# Blinkreconstruction

(One of) the most interesting parts of the eyelinkparser is this:

```
dm = parse(
        traceprocessor = defaulttraceprocessor(
          blinkreconstruct = True,
          mode= 'advanced',
          downsample = None
        )
    )
```

In our analysis, blink reconstruction is disabled. During blinks, data on pupil size is unavailable. To address this, the parser can impute the missing data based on established methods [reference needed]. The procedure involves two main steps:

1. Correction of Blink-Affected Data: The time series data immediately before and after a blink, which may be influenced by the blink (due to the closing and reopening of the eyes), are adjusted to mitigate any artifacts caused by the blink. This ensures that the data reflects the true pupil size changes rather than the mechanical effects of blinking.

1. Interpolation During Blinks: The time series data during the blink itself, where data is completely missing, is interpolated. This interpolation fills in the missing data points in a way that maintains the continuity and trend of the pupil size data, providing a seamless transition across the blink period.

This advanced blink reconstruction technique ensures that the pupil size data is as accurate and reliable as possible, minimizing the impact of blinks on the overall analysis.



When a pp blinks, data recorded during that period is missing or distorted.

We use srs.blinkreconstruct to create a new trace in which these are restored. 

For now, we'll call the column "pupil_reconstructed". In practice, you would overwrite the original one, or use a more practical (i.e., short) name.


In [ ]:
dm.pupil_reconstructed = srs.blinkreconstruct(dm.pupil, mode = "advanced")

Lets visualise how that reconstruction works.

We can plot the two traces simultaniously. We'll color the original one red ("alpha = .5" will also make it a bit "see through"). In most trials, the pp will not have blinked. See if those blinks you found in the previous assignment were interpolated correctly. 


We use MatPlotLib for plotting here. There is sometimes an issue with the fonts matplotlib defaults to. Hopefully this is a fix for that. If you have many warnings about fonts, please let me know...

In [ ]:
import matplotlib as mpl
import matplotlib.font_manager
matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

mpl.rcParams['font.family'] = "sans-serif"
mpl.rcParams['font.sans-serif'] = "DejaVu Sans"

In [ ]:
row_nr = 4
plt.figure()
plt.plot(dm.pupil_reconstructed[row_nr], color = "black")
plt.plot(dm.pupil[row_nr], color = "red", alpha = .5)
plt.title(row_nr)



srs.blinkreconstruction accepts a number of parameters that might change its functioning. 

ASSIGNMENT:

   In your own words, describe what the following do:
    
    - vt

#TODO

    - margin

#TODO

    - smooth_winlen 

#TODO

### Overwrite raw pupil data
Now we overwrite our raw pupil column with the reconstructed one

In [ ]:
dm.pupil = dm.pupil_reconstructed

## Data Cleaning

In this example, the data cleaning is automated. This is not always possible, and therefore this placeholder title is inserted. If needed: Clean your data here.

## Baseline Correction

As we are interested in the *reaction* of the pupil, we start with a baseline correction. The period just before our period of interest is defined as a baseline, and set to 0. The change from this baseline now is the variable we are testing and plotting.

As a baseline, we could use the last ten samples of the basline trace. The period before the stimuluspresentation is 1500 ms. If we want the baseline to be only 10 ms, we can define this as follows:


In [ ]:
dm.pupil = srs.baseline(
    series=dm.pupil,
    baseline=dm.pupil,
    bl_start=1490,
    bl_end=1500
)

## Data Visualization

This section provides methods for visualizing the data to gain insights.

### Saving plots as pngs

# Individual Traces

Plot all different traces into one single plot. This will reveal problems in your data.

In [ ]:
for row in dm:    
    plt.plot(row.pupil, alpha = .1, color = "black")
plt.ylim(-500, 500)
plt.savefig('plots/all_trials.png', bbox_inches='tight')

Note the "bow-tie" shape of the plot. Why is that?

# Grand averages

Plot the overall average pupil size during the trials:

In [ ]:
plt.plot(dm.pupil[0], color = "black")
plt.ylim(-500, 500)
plt.savefig('plots/single_trial.png', bbox_inches='tight')

# Testing the effects


reference: https://github.com/smathot/time_series_test


# Plotting the experimental effects:

The *time_series_test* package (also) has easy plotting functions.

In [ ]:
import time_series_test as tst
from matplotlib import pyplot as plt

tst.plot(dm, dv='pupil', hue_factor='difficulty')
plt.savefig('plots/signal-plot-1.png')

The model below uses mean pupil size during the 2500 - 3000 sample range as dependent measure, set size and color type as fixed effects, and a random by-subject intercept. In the more familiar notation of the R package lme4, this corresponds to ```mean_pupil ~ difficulty + (1 | subject_nr)```. (To use more complex random-effects structures, you can use the re_formula argument to mixedlm().)

In [ ]:
from statsmodels.formula.api import mixedlm
from datamatrix import series as srs, NAN

dm.mean_pupil = srs.reduce(dm.pupil[:, 2500:3500])
dm_valid_data = dm.mean_pupil != NAN
model = mixedlm(formula='mean_pupil ~ difficulty',
                data=dm_valid_data, groups='subject_nr').fit()
print(model.summary())